In [1]:
import pandas as pd
from ast import literal_eval

data = pd.read_csv('spotify_data/data.csv')
data['artists'] = data['artists'].apply(literal_eval)
data_w_genres = pd.read_csv('spotify_data/data_w_genres.csv')

In [2]:
artist_genres = data_w_genres[['artists', 'genres']]
separated_artists = data.explode('artists')
separated_artists = separated_artists.join(artist_genres.set_index('artists'), on='artists')

cols = set(separated_artists.columns)
cols.remove('artists')
cols.remove('genres')
songs_w_genres = separated_artists.groupby(list(cols), as_index=False).agg({'artists': [list], 'genres': [lambda x: x.iloc[0]]})
songs_w_genres.columns = songs_w_genres.columns.droplevel(-1)


In [3]:
useless_columns = ['artists', 'year', 'release_date', 'id', 'name']
songs_w_genres = songs_w_genres.drop(useless_columns, axis=1)

songs_with_genres = songs_w_genres[songs_w_genres.genres != '[]'].dropna()

songs_with_genres.to_csv('spotify_data/processed_data.csv', index=False)